In [1]:
import numpy as np
import random  # for the random wirings
import time # to compare the algorithm timings

> See the attached PDF for the meaning of the notations.

# I. Vectorialization

## 1.1. Definition of h

In [178]:
"""CHSH = np.array([
    [0.25, 0, 0, 0.25],
    [0.25, 0, 0, 0.25],
    [0.25, 0, 0, 0.25],
    [0, 0.25, 0.25, 0]
])"""

CHSH = np.zeros((4,4))

for a in range(2):
    for b in range(2):
        for x in range(2):
            for y in range(2):
                if (a+b)%2 == x*y:
                    CHSH[2*x+y, 2*a+b]=0.25
                    
print(CHSH)

[[0.25 0.   0.   0.25]
 [0.25 0.   0.   0.25]
 [0.25 0.   0.   0.25]
 [0.   0.25 0.25 0.  ]]


In [179]:
def h(Q):
    return np.dot( CHSH.flatten(), Q.flatten() )  # scalar product of Q and CHSH

In [183]:
PR = np.zeros((4,4))
for a in range(2):
    for b in range(2):
        for x in range(2):
            for y in range(2):
                if (a+b)%2 == x*y:
                    PR[2*x+y, 2*a+b]=0.5
                

SR = np.zeros((4,4))
for a in range(2):
    for b in range(2):
        for x in range(2):
            for y in range(2):
                if a==b:
                    SR[2*x+y, 2*a+b]=0.5

In [184]:
h(PR)

1.0

In [185]:
h(SR)

0.75

## 1.2. Computation of A(W)

In [23]:
# A1 is a 2x4x4x32-tensor
A1 = np.zeros( (2, 4, 4, 32) )

for x in range(2):
    for j in range(4):
        
        sign = 1
        if j >= 2:
            sign=-1
            
        A1[x, 0, j, 0] = sign*(x-1)
        A1[x, 1, j, 0] = sign*(x-1)
        A1[x, 2, j, 1] = sign*(x-1)
        A1[x, 3, j, 1] = sign*(x-1)
        
        A1[x, 0, j, 2] = sign*(-x)
        A1[x, 1, j, 2] = sign*(-x)
        A1[x, 2, j, 3] = sign*(-x)
        A1[x, 3, j, 3] = sign*(-x)

#print(A1)

In [24]:
# A2 is a 2x4x4-tensor
A2 = np.zeros( (2, 4, 4) )

for x in range(2):
    for i in range(4):
        for k in range(4):
            if k<=1:
                A2[x, i, k]=1

#print(A2)

In [22]:
W_BS09 = [0, 0, 1, 1,              # f_1(x, a_2) = x
          0, 0, 1, 1,              # g_1(y, b_2) = y
          0, 0, 0, 1,              # f_2(x, a_1) = a_1*x
          0, 0, 0, 1,              # g_2(y, b_1) = b_1*y
          0, 1, 1, 0, 0, 1, 1, 0,  # f_3(x, a_1, a_2) = a_1 + a_2 mod 2
          0, 1, 1, 0, 0, 1, 1, 0   # g_3(y, b_1, b_2) = b_1 + b_2 mod 2
         ]

np.tensordot(A1, W_BS09, axes=([3, 0])) + A2

array([[[1., 1., 0., 0.],
        [1., 1., 0., 0.],
        [1., 1., 0., 0.],
        [1., 1., 0., 0.]],

       [[0., 0., 1., 1.],
        [0., 0., 1., 1.],
        [0., 0., 1., 1.],
        [0., 0., 1., 1.]]])

-> This is well the wanted result

In [26]:
# A3 is a 2x4x4x32-tensor
A3 = np.zeros( (2, 4, 4, 32) )

for y in range(2):
    for j in range(4):
        
        sign = 1
        if j==1 or j==3:
            sign=-1
            
        A3[y, 0, j, 0 +4] = sign*(y-1)
        A3[y, 2, j, 0 +4] = sign*(y-1)
        A3[y, 1, j, 1 +4] = sign*(y-1)
        A3[y, 3, j, 1 +4] = sign*(y-1)
        
        A3[y, 0, j, 2 +4] = sign*(-y)
        A3[y, 2, j, 2 +4] = sign*(-y)
        A3[y, 1, j, 3 +4] = sign*(-y)
        A3[y, 3, j, 3 +4] = sign*(-y)
        
#print(A3)

In [28]:
# A4 is a 2x4x4-tensor
A4 = np.zeros( (2, 4, 4) )

for y in range(2):
    for i in range(4):
        for k in range(4):
            if k==0 or k==2:
                A4[y, i, k]=1
                
#print(A4)

In [29]:
np.tensordot(A3, W_BS09, axes=([3, 0])) + A4

array([[[1., 0., 1., 0.],
        [1., 0., 1., 0.],
        [1., 0., 1., 0.],
        [1., 0., 1., 0.]],

       [[0., 1., 0., 1.],
        [0., 1., 0., 1.],
        [0., 1., 0., 1.],
        [0., 1., 0., 1.]]])

-> We well obtain the wanted result.

We obtain $A(W)$:

In [113]:
def A(W):  # W is a vector with 32 entries
    T1 = np.tensordot(A1, W, axes=([3, 0])) + A2
    T2 = np.tensordot(np.ones((2)), T1, axes=0)
    T3 = np.transpose(T2, (1,0,2,3))
    S1 = np.tensordot(A3, W, axes=([3, 0])) + A4
    S2 = np.tensordot(np.ones((2)), S1, axes=0)
    return T3 * S2

A(W_BS09).shape
    

(2, 2, 4, 4)

-> This is the wanted result.

## 1.3. Definition of B(W)

In [34]:
# B1 is a 2x4x4x32-tensor
B1 = np.zeros( (2, 4, 4, 32) )

for x in range(2):
    for l in range(4):
        
        sign = 1
        if l>=2:
            sign=-1
            
        B1[x, 0, l, 0 +8] = sign * (x-1)
        B1[x, 1, l, 0 +8] = sign * (x-1)
        B1[x, 2, l, 1 +8] = sign * (x-1)
        B1[x, 3, l, 1 +8] = sign * (x-1)
        
        B1[x, 0, l, 2 +8] = sign * (-x)
        B1[x, 1, l, 2 +8] = sign * (-x)
        B1[x, 2, l, 3 +8] = sign * (-x)
        B1[x, 3, l, 3 +8] = sign * (-x)

#print(B1)

In [36]:
# B2 is equal to A2
B2 = A2

#print(B2)

In [38]:
# B3 is a 2x4x4x32-tensor
B3 = np.zeros( (2, 4, 4, 32) )

for y in range(2):
    for l in range(4):
        
        sign=1
        if l==1 or l==3:
            sign=-1
        
        B3[y, 0, l, 0 +12] = sign * (y-1)
        B3[y, 2, l, 0 +12] = sign * (y-1)
        B3[y, 1, l, 1 +12] = sign * (y-1)
        B3[y, 3, l, 1 +12] = sign * (y-1)
        
        B3[y, 0, l, 2 +12] = sign * (-y)
        B3[y, 2, l, 2 +12] = sign * (-y)
        B3[y, 1, l, 3 +12] = sign * (-y)
        B3[y, 3, l, 3 +12] = sign * (-y)

#print(B3)

In [40]:
# B4 is equal to A4
B4 = A4

#print(B4)

In [115]:
def B(W):   # W is a vector with 32 entries
    T1 = np.tensordot(B1, W, axes=([3, 0])) + B2
    T2 = np.tensordot(np.ones((2)), T1, axes=0)
    T3 = np.transpose(T2, (1,0,2,3))
    S1 = np.tensordot(B3, W, axes=([3, 0])) + B4
    S2 = np.tensordot(np.ones((2)), S1, axes=0)
    return T3 * S2

B(W_BS09).shape


(2, 2, 4, 4)

-> This is the wanted result.

## 1.4. Definition of C(W)

In [94]:
# C1 is a 2x2x4x4x32-tensor
C1 = np.zeros( (2, 2, 4, 4, 32) )

for a in range(2):
    for x in range(2):
        for j in range(4):
            if j<=1:
                C1[a, x, 0, j, 0 +16] = -(1-x) * (-1)**a
                C1[a, x, 1, j, 0 +16] = -(1-x) * (-1)**a
                C1[a, x, 2, j, 1 +16] = -(1-x) * (-1)**a
                C1[a, x, 3, j, 1 +16] = -(1-x) * (-1)**a
                
                C1[a, x, 0, j, 4 +16] = -(x) * (-1)**a
                C1[a, x, 1, j, 4 +16] = -(x) * (-1)**a
                C1[a, x, 2, j, 5 +16] = -(x) * (-1)**a
                C1[a, x, 3, j, 5 +16] = -(x) * (-1)**a
            
            if j>=2:
                C1[a, x, 0, j, 0 +18] = -(1-x) * (-1)**a
                C1[a, x, 1, j, 0 +18] = -(1-x) * (-1)**a
                C1[a, x, 2, j, 1 +18] = -(1-x) * (-1)**a
                C1[a, x, 3, j, 1 +18] = -(1-x) * (-1)**a
                
                C1[a, x, 0, j, 4 +18] = -(x) * (-1)**a
                C1[a, x, 1, j, 4 +18] = -(x) * (-1)**a
                C1[a, x, 2, j, 5 +18] = -(x) * (-1)**a
                C1[a, x, 3, j, 5 +18] = -(x) * (-1)**a
                

#print(C1)

In [95]:
np.tensordot(C1, W_BS09, axes=([4, 0]))

array([[[[ 0.,  0., -1., -1.],
         [ 0.,  0., -1., -1.],
         [-1., -1.,  0.,  0.],
         [-1., -1.,  0.,  0.]],

        [[ 0.,  0., -1., -1.],
         [ 0.,  0., -1., -1.],
         [-1., -1.,  0.,  0.],
         [-1., -1.,  0.,  0.]]],


       [[[ 0.,  0.,  1.,  1.],
         [ 0.,  0.,  1.,  1.],
         [ 1.,  1.,  0.,  0.],
         [ 1.,  1.,  0.,  0.]],

        [[ 0.,  0.,  1.,  1.],
         [ 0.,  0.,  1.,  1.],
         [ 1.,  1.,  0.,  0.],
         [ 1.,  1.,  0.,  0.]]]])

-> This is well the expected result.

In [97]:
# C2 is a 2x2x4x4-tensor
C2 = np.zeros( (2, 2, 4, 4) )

for x in range(2):
    for i in range(4):
        for j in range(4):
            C2[0, x, i, j]=1
            
#print(C2)

---
#### Kronecker product:

In [136]:
v = np.arange(4).reshape(2,2)
w = np.ones((3, 3))

print(np.tensordot(w, v, axes=0))

[[[[0. 1.]
   [2. 3.]]

  [[0. 1.]
   [2. 3.]]

  [[0. 1.]
   [2. 3.]]]


 [[[0. 1.]
   [2. 3.]]

  [[0. 1.]
   [2. 3.]]

  [[0. 1.]
   [2. 3.]]]


 [[[0. 1.]
   [2. 3.]]

  [[0. 1.]
   [2. 3.]]

  [[0. 1.]
   [2. 3.]]]]


#### Permute some entries of a tensor:

In [89]:
x = np.ones((480, 640, 3, 42))
np.transpose(x, (1, 3, 2, 0)).shape  # the new 0 is the old 1
                                     # the new 1 is the old 3
                                     # 2 stays 2
                                     # the new 3 is the old 0

(640, 42, 3, 480)

---

In [104]:
def C(W):    # W is a vector with 32 entries
    T1 = np.tensordot(C1, W, axes=([4, 0])) + C2
    T2 = np.tensordot( np.ones((2,2)), T1, axes=0)  # Kronecker product
    return np.transpose(T2, (2, 0, 3, 1, 4, 5))

C(W_BS09)

array([[[[[[1., 1., 0., 0.],
           [1., 1., 0., 0.],
           [0., 0., 1., 1.],
           [0., 0., 1., 1.]],

          [[1., 1., 0., 0.],
           [1., 1., 0., 0.],
           [0., 0., 1., 1.],
           [0., 0., 1., 1.]]],


         [[[1., 1., 0., 0.],
           [1., 1., 0., 0.],
           [0., 0., 1., 1.],
           [0., 0., 1., 1.]],

          [[1., 1., 0., 0.],
           [1., 1., 0., 0.],
           [0., 0., 1., 1.],
           [0., 0., 1., 1.]]]],



        [[[[1., 1., 0., 0.],
           [1., 1., 0., 0.],
           [0., 0., 1., 1.],
           [0., 0., 1., 1.]],

          [[1., 1., 0., 0.],
           [1., 1., 0., 0.],
           [0., 0., 1., 1.],
           [0., 0., 1., 1.]]],


         [[[1., 1., 0., 0.],
           [1., 1., 0., 0.],
           [0., 0., 1., 1.],
           [0., 0., 1., 1.]],

          [[1., 1., 0., 0.],
           [1., 1., 0., 0.],
           [0., 0., 1., 1.],
           [0., 0., 1., 1.]]]]],




       [[[[[0., 0., 1., 1.],
           [0

-> It seems good

## 1.5. Definition of D(W)

In [100]:
# D1 is a 2x2x4x4x32-tensor
D1 = np.zeros( (2, 2, 4, 4, 32) )

for b in range(2):
    for y in range(2):
        for j in range(4):
            if j==0 or j==2:
                D1[b, y, 0, j, 0 + 24] = -(1-y) * (-1)**b
                D1[b, y, 2, j, 0 + 24] = -(1-y) * (-1)**b
                D1[b, y, 1, j, 1 + 24] = -(1-y) * (-1)**b
                D1[b, y, 3, j, 1 + 24] = -(1-y) * (-1)**b
                
                D1[b, y, 0, j, 4 + 24] = -(y) * (-1)**b
                D1[b, y, 2, j, 4 + 24] = -(y) * (-1)**b
                D1[b, y, 1, j, 5 + 24] = -(y) * (-1)**b
                D1[b, y, 3, j, 5 + 24] = -(y) * (-1)**b
            
            if j==1 or j==3:
                D1[b, y, 0, j, 0 + 26] = -(1-y) * (-1)**b
                D1[b, y, 2, j, 0 + 26] = -(1-y) * (-1)**b
                D1[b, y, 1, j, 1 + 26] = -(1-y) * (-1)**b
                D1[b, y, 3, j, 1 + 26] = -(1-y) * (-1)**b
                
                D1[b, y, 0, j, 4 + 26] = -(y) * (-1)**b
                D1[b, y, 2, j, 4 + 26] = -(y) * (-1)**b
                D1[b, y, 1, j, 5 + 26] = -(y) * (-1)**b
                D1[b, y, 3, j, 5 + 26] = -(y) * (-1)**b

#print(D1)

In [101]:
np.tensordot(D1, W_BS09, axes=([4, 0]))

array([[[[ 0., -1.,  0., -1.],
         [-1.,  0., -1.,  0.],
         [ 0., -1.,  0., -1.],
         [-1.,  0., -1.,  0.]],

        [[ 0., -1.,  0., -1.],
         [-1.,  0., -1.,  0.],
         [ 0., -1.,  0., -1.],
         [-1.,  0., -1.,  0.]]],


       [[[ 0.,  1.,  0.,  1.],
         [ 1.,  0.,  1.,  0.],
         [ 0.,  1.,  0.,  1.],
         [ 1.,  0.,  1.,  0.]],

        [[ 0.,  1.,  0.,  1.],
         [ 1.,  0.,  1.,  0.],
         [ 0.,  1.,  0.,  1.],
         [ 1.,  0.,  1.,  0.]]]])

-> This is well the wanted result.

In [103]:
# D2 is a 2x2x4x4-tensor
D2 = np.zeros( (2, 2, 4, 4) )

for y in range(2):
    for i in range(4):
        for j in range(4):
            D2[0, y, i, j] = 1

#print(D2)

In [105]:
def D(W):    # W is a vector with 32 entries
    T1 = np.tensordot(D1, W, axes=([4, 0])) + D2
    T2 = np.tensordot( np.ones((2,2)), T1, axes=0)  # Kronecker product
    return np.transpose(T2, (0, 2, 1, 3, 4, 5))

D(W_BS09)

array([[[[[[1., 0., 1., 0.],
           [0., 1., 0., 1.],
           [1., 0., 1., 0.],
           [0., 1., 0., 1.]],

          [[1., 0., 1., 0.],
           [0., 1., 0., 1.],
           [1., 0., 1., 0.],
           [0., 1., 0., 1.]]],


         [[[1., 0., 1., 0.],
           [0., 1., 0., 1.],
           [1., 0., 1., 0.],
           [0., 1., 0., 1.]],

          [[1., 0., 1., 0.],
           [0., 1., 0., 1.],
           [1., 0., 1., 0.],
           [0., 1., 0., 1.]]]],



        [[[[0., 1., 0., 1.],
           [1., 0., 1., 0.],
           [0., 1., 0., 1.],
           [1., 0., 1., 0.]],

          [[0., 1., 0., 1.],
           [1., 0., 1., 0.],
           [0., 1., 0., 1.],
           [1., 0., 1., 0.]]],


         [[[0., 1., 0., 1.],
           [1., 0., 1., 0.],
           [0., 1., 0., 1.],
           [1., 0., 1., 0.]],

          [[0., 1., 0., 1.],
           [1., 0., 1., 0.],
           [0., 1., 0., 1.],
           [1., 0., 1., 0.]]]]],




       [[[[[1., 0., 1., 0.],
           [0

-> It seems good!

## 1.6. Product of Boxes $R(W, P, Q) = P \boxtimes_W Q$

In [147]:
def R(W, P, Q):
    T1 = np.tensordot(A(W), P, axes=([3, 0]))  # green term
    T2 = np.transpose(np.tensordot(B(W), Q, axes=([3, 0])), (0,1,3,2))  # blue term
    T3 = np.tensordot(np.ones((2,2)), T1*T2, axes = 0)   # Kronecker product
    T4 = T3 * C(W) * D(W)  # the big bracket
    T5 = np.tensordot(T4, np.ones((4)), axes=([5, 0]))
    return np.tensordot(T5, np.ones((4)), axes = ([4,0]))
    
    

In [149]:
R(W_BS09, PR, SR)

array([[[[0.5, 0.5],
         [0.5, 0. ]],

        [[0. , 0. ],
         [0. , 0.5]]],


       [[[0. , 0. ],
         [0. , 0.5]],

        [[0.5, 0.5],
         [0.5, 0. ]]]])

## 1.7. Write R as a matrix

In [164]:
def tensor_to_matrix(Tensor):
    Matrix = np.zeros((4,4))
    for a in range(2):
        for b in range(2):
            for x in range(2):
                for y in range(2):
                    Matrix[2*x+y, 2*a+b] = Tensor[a, b, x, y]
    return Matrix

In [165]:
def matrix_to_tensor(Matrix):
    Tensor = np.zeros((2,2,2,2))
    for i in range(4):
        for j in range(4):
            b = j%2
            a = (j-b)//2
            y = i%2
            x = (i-y)//2
            Tensor[a,b,x,y] = Matrix[i,j]
    return Tensor

In [168]:
tensor_to_matrix(matrix_to_tensor(PR)) == PR

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

In [169]:
tensor_to_matrix(matrix_to_tensor(SR)) == SR

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

In [171]:
matrix_to_tensor(tensor_to_matrix(R(W_BS09, PR, SR))) == R(W_BS09, PR, SR)

array([[[[ True,  True],
         [ True,  True]],

        [[ True,  True],
         [ True,  True]]],


       [[[ True,  True],
         [ True,  True]],

        [[ True,  True],
         [ True,  True]]]])

In [173]:
tensor_to_matrix(R(W_BS09, PR, SR)) == PR

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

In [187]:
tensor_to_matrix(R(W_BS09, PR, PR)) == PR

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

## 1.8. Function to maximize

In [189]:
def phi(W, P, Q):
    # W is a vector in R^32
    # P is a box: a 4x4-matrix
    # Q is a box: a 4x4-matrix
    
    return h(tensor_to_matrix(R(W,P,Q)))

In [191]:
phi(W_BS09, PR, PR)

1.0

## 1.9. Some other boxes

In [192]:
def P_L(mu, nu, sigma, tau):
    new_box = np.zeros((4,4))
    
    for a in range(2):
        for b in range(2):
            for x in range(2):
                for y in range(2):
                    if a==(mu*x+nu)%2 and b==(sigma*y+tau)%2:
                        new_box[2*x+y, 2*a+b] = 1
                        
    return new_box
                        

In [193]:
P_0 = P_L(0,0,0,0)
P_1 = P_L(0,1,0,1)

In [194]:
def P_NL(mu, nu, sigma):
    new_box = np.zeros((4,4))
    
    for a in range(2):
        for b in range(2):
            for x in range(2):
                for y in range(2):
                    if (a+b)%2==(x*y + mu*x + nu*y + sigma)%2:
                        new_box[2*x+y, 2*a+b] = 0.5
                        
    return new_box

In [195]:
P_NL(0,0,0) == PR

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

In [214]:
PRbar = P_NL(0,0,1)
I = 0.25*np.ones((4,4))
SRbar = 2*I-SR
(PRbar/2 + PR/2)==I

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

In [197]:
def corNLB(p):
    return p*PR + (1-p)*SR

In [204]:
p=random.random()

h(corNLB(p)) == (p+3)/4

True

In [205]:
p=random.random()

tensor_to_matrix(R(W_BS09, PR, corNLB(p))) == PR

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

In [209]:
p=random.random()

tensor_to_matrix(R(W_BS09, corNLB(p), PR)) == p*PR + (1-p)*(PR+SR)/2

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

In [215]:
p=random.random()

tensor_to_matrix(R(W_BS09, corNLB(p), PRbar)) == p*PRbar + (1-p)*(PRbar+SRbar)/2


array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

In [216]:
p=random.random()

tensor_to_matrix(R(W_BS09, corNLB(p), SRbar)) == p*PRbar + (1-p)*(SRbar)


array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

## 1.10. Some other wirings

In [ ]:
central_wiring = [0.5 for _ in range(32)]

def random_wiring():
    return [random.random() for _ in range(32)]

In [ ]:
W_BS09 = [0, 0, 1, 1,              # f_1(x, a_2) = x
          0, 0, 1, 1,              # g_1(y, b_2) = y
          0, 0, 0, 1,              # f_2(x, a_1) = a_1*x
          0, 0, 0, 1,              # g_2(y, b_1) = b_1*y
          0, 1, 1, 0, 0, 1, 1, 0,  # f_3(x, a_1, a_2) = a_1 + a_2 mod 2
          0, 1, 1, 0, 0, 1, 1, 0   # g_3(y, b_1, b_2) = b_1 + b_2 mod 2
         ]

W_ABLPSV09 = [
          0, 0, 1, 1,              # f_1(x, a_2) = x
          0, 0, 1, 1,              # g_1(y, b_2) = y
          1, 0, 0, 1,              # f_2(x, a_1) = x ⊕ a_1 ⊕ 1
          0, 0, 0, 1,              # g_2(y, b_1) = y*b_1
          1, 0, 0, 1, 1, 0, 0, 1,  # f_3(x, a_1, a_2) = a_1 ⊕ a_2 ⊕ 1
          1, 0, 0, 1, 1, 0, 0, 1,  # g_3(y, b_1, b_2) = b_1 ⊕ b_2 ⊕ 1
]


def wiring_to_functions(W):
    print("f_1(x,a2) = ", (W[2]-W[0])%2, "x ⊕ ", (W[1]-W[0])%2 ,"a2 ⊕ ", (W[3]-W[2]-W[1]+W[0])%2 ,"x*a2 ⊕ ", (W[0])%2)
    print("g_1(y,b2) = ", (W[6]-W[4])%2, "y ⊕ ", (W[5]-W[4])%2 ,"b2 ⊕ ", (W[7]-W[6]-W[5]+W[4])%2 ,"y*b2 ⊕ ", (W[4])%2)
    print("f_2(x,a1) = ", (W[10]-W[8])%2, "x ⊕ ", (W[9]-W[8])%2 ,"a1 ⊕ ", (W[11]-W[10]-W[9]+W[8])%2 ,"x*a1 ⊕ ", (W[8])%2)
    print("g_2(y,b1) = ", (W[14]-W[12])%2, "y ⊕ ", (W[13]-W[12])%2 ,"b1 ⊕ ", (W[15]-W[14]-W[13]+W[12])%2 ,"y*b1 ⊕ ", (W[12])%2)
    print("f_3(x,a1,a2) = ", (W[20]-W[16])%2, "x ⊕ ", (W[18]-W[16])%2 ,"a1 ⊕ ", (W[17]-W[16])%2 ,"a2 ⊕ ", 
          (W[21]-W[20]-W[18]+W[16])%2 ,"x*a1 ⊕ ", (W[22]-W[20]-W[17]+W[16])%2 ,"x*a2 ⊕ ", 
          (W[19]-W[18]-W[17]+W[16])%2 ,"a1*a2 ⊕ ", 
          (W[23] - W[21] - W[22]+W[20] - W[19]+W[18]+W[17] - W[16])%2,"x*a1*a2 ⊕ ", (W[16])%2)
    print("g_3(y,b1,b2) = ", (W[28]-W[24])%2, "y ⊕ ", (W[26]-W[24])%2 ,"b1 ⊕ ", (W[25]-W[24])%2 ,"b2 ⊕ ", 
          (W[29]-W[28]-W[26]+W[24])%2 ,"y*b1 ⊕ ", (W[30]-W[28]-W[25]+W[24])%2 ,"y*b2 ⊕ ", 
          (W[27]-W[26]-W[25]+W[24])%2 ,"b1*b2 ⊕ ", 
          (W[31] - W[29] - W[30]+W[28] - W[27]+W[26]+W[25] - W[24])%2,"y*b1*b2 ⊕ ", (W[16])%2)